# Qamomile v0.8.0

---

This release mayjorly encences the accessibility for broader people by adding some [tutorials in Japanese](https://jij-inc.github.io/Qamomile/ja/), usability by supporting [OMMX v2](https://jij-inc.github.io/ommx/en/introduction.html) and introducing a new target quantum SDK, [CUDA-Q](https://developer.nvidia.com/cuda-q), as well as [Bloqade-Analog](https://github.com/QuEraComputing/bloqade-analog). We believe that, especially OMMX v2 support is a big step for us and users as OMMX allows us to compare quantum optimisation algorithms and classical solvers esaier than before as OMMX provides us the united form of optimisation problems.

We have other updates having slightly smaller impacts as well. Please have a look at our [releases](https://github.com/Jij-Inc/Qamomile/releases) and individual PRs for more details.

## ✨ New Features


### OMMX v2 Support

### CUDA-Q Support

### Bloqade-Analog

## 🐛 Bug Fixes

See our [releases](https://github.com/Jij-Inc/Qamomile/releases) for details.

## 🛠️ Other Changes and Improvements

See our [releases](https://github.com/Jij-Inc/Qamomile/releases) for details.

## 💬 Feedback
With these new features, Qamomile becomes a powerful tool for enabling users to access several quantum SDKs in a same way and compare the results with classical solvers. Try out Qamomile 0.8.0!


Please submit any feedback or bug reports to [GitHub Issues](https://github.com/Jij-Inc/Qamomile/issues).